# Barotropic Vorticity Budget

In this notebook, we compute barotropic vorticity budget terms using the depth-integrated momentum budget diagnostics from the CM4X model. The details can be found in [Khatri et al. (2024)](https://doi.org/10.1029/2023MS003813) and https://github.com/hmkhatri/Barotropic_Vorticity_Analysis_GCM. 

In a non steady state, the following balance holds

\begin{equation}
\beta {\int_{-H}^{\eta}vdz} = \dfrac{1}{\rho_o}{J(p_b, H)} - {\dfrac{fQ_m}{\rho_o}} + f\partial_t\eta - \nabla \wedge \int_{-H}^{\eta} \mathbf{u}\,dz + \dfrac{1}{\rho_o}\nabla \wedge{\left(\boldsymbol{\tau_s - \tau_b}\right)} + {\nabla \wedge \int_{-H}^{\eta}\mathbf{a}\,dz} + {\nabla \wedge \int_{-H}^{\eta}\mathbf{b}\,dz} \tag{1}
\end{equation}

\begin{equation}
\beta \, {V} = \dfrac{1}{\rho_o}{J(p_b, H)} - f{\dfrac{Q_m}{\rho_o}} + f\partial_t\eta - \nabla \wedge \mathcal{U}_t + \dfrac{1}{\rho_o}\nabla \wedge{\left(\boldsymbol{\tau_s - \tau_b}\right)} + {\nabla \wedge \mathcal{A}} + {\nabla \wedge \mathcal{B}} \tag{2}
\end{equation}

It is seen that, in addition to boundary stress terms (fifth term on the RHS), bottom pressure torque (first term on the RHS) and nonlinear terms (sixth term on the RHS) do contribute significantly to the overall vorticity baalnce. Here, we isolate the relative contributions of these terms.

In order to compute bottom pressure torque, we use the following relation (the terms on the RHS are available as diagnostics)

\begin{equation}
\dfrac{1}{\rho_o}{J(p_b, H)} = \nabla \wedge\left[- f \hat{{\bf z}} \wedge \int_{-H}^{\eta} \mathbf{u}\,dz - \frac{1}{\rho_o}\int_{-H}^{\eta} \nabla p\,dz \right] + \beta \, {V} + f{\dfrac{Q_m}{\rho_o}} - f\partial_t\eta \tag{3}
\end{equation}



In [1]:
import xarray as xr
import numpy as np
from xgcm import Grid
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import glob, os

In [2]:
def Vorticity_Budget(ds, ds_grid):

    """Compute barotropic vorticity budget terms
    Parameters
    ----------
    ds : xarray Dataset for depth-integrated momentum budget diagnostics and grid information
    
    Returns
    -------
    ds_save : Dataset containting barotropic vorticity budget terms
    """

    # Create grid information and compute beta

    OMEGA = 7.2921e-5
    RAD_EARTH = 6.378e6,
    rho_0 = 1035.
    
    grid = Grid(ds, coords={'X': {'center': 'xh', 'right': 'xq'},
                            'Y': {'center': 'yh', 'right': 'yq'}}, periodic=['X'])
    
    colh_u = grid.interp(ds['col_height'] * ds['areacello'], 'X',  boundary='fill') / ds['areacello_cu']
    colh_v = grid.interp(ds['col_height'] * ds['areacello'], 'Y',  boundary='fill') / ds['areacello_cv']
    
    beta_v = 2*OMEGA*np.cos(ds.geolat_v * np.pi /180.)/RAD_EARTH
    beta_q = 2*OMEGA*np.cos(ds.geolat_c * np.pi /180.)/RAD_EARTH

    # compute terms in vorticity budget
    
    BPT = xr.Dataset()

    vmo_bv = (ds['vmo_2d'] / (rho_0 * ds['dxCv']))
    vmo_bv = beta_q * grid.interp(vmo_bv, 'X',  boundary='fill')
    BPT['vmo_bv'] = vmo_bv
    
    umo = (ds['umo_2d'] / (rho_0 * ds['dyCu']))
    umo = grid.interp(umo, 'Y',  boundary='fill')
    BPT['umo'] = umo
    
    BPT_1 = (( - grid.diff((ds['intz_PFu_2d'] + ds['intz_u_BT_accel_2d']) * ds.dxCu, 'Y', boundary='fill')
               + grid.diff((ds['intz_PFv_2d'] + ds['intz_v_BT_accel_2d']) * ds.dyCv, 'X', boundary='fill')) 
             / ds.areacello_bu)
    BPT['BPT'] = BPT_1
    
    Mass_Surf = (grid.interp(grid.interp(ds['wfo'] * ds['areacello'], 'X', boundary='fill'), 'Y',  boundary='fill') 
                 * ds['Coriolis'] / (rho_0)) / ds['areacello_bu']
    BPT['Qm'] = Mass_Surf
    
    div_u = (grid.diff(ds['umo_2d'] / (rho_0), 'X', boundary='fill') + 
             grid.diff(ds['vmo_2d'] / (rho_0), 'Y', boundary='fill') ) / ds['areacello']
    div_u = (grid.interp(grid.interp(div_u * ds['areacello'], 'X', boundary='fill'), 'Y', boundary='fill')
            * ds['Coriolis']) / ds['areacello_bu']
    BPT['div_u'] = div_u
    
    BPT['fdhdt'] = (Mass_Surf - div_u)
    
    Curl_dudt = ( - grid.diff(ds['hf_dudt_2d'] * colh_u * ds['dxCu'], 'Y', boundary='fill')
                + grid.diff(ds['hf_dvdt_2d'] * colh_v * ds['dyCv'], 'X', boundary='fill') ) / ds.areacello_bu
    BPT['Curl_dudt'] = Curl_dudt
    
    Curl_taus = ( - grid.diff((ds['taux'])* ds.dxCu, 'Y', boundary='fill')
                       + grid.diff((ds['tauy'])* ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
    Curl_taus = Curl_taus / (rho_0 )
    BPT['Curl_taus'] = Curl_taus
    
    Curl_taub = ( - grid.diff((-ds['taux_bot'])* ds.dxCu, 'Y', boundary='fill')
                       + grid.diff(-ds['tauy_bot'] * ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
    Curl_taub = Curl_taub / (rho_0 )
    BPT['Curl_taub'] = Curl_taub
    
    Curl_Hrv2 = ( - grid.diff((ds['intz_rvxv_2d'] + ds['intz_gKEu_2d']) * ds.dxCu, 'Y', boundary='fill')
               + grid.diff((ds['intz_rvxu_2d'] + ds['intz_gKEv_2d']) * ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
    BPT['Curl_NL'] = Curl_Hrv2
    
    Curl_Hdiff2 = ( - grid.diff(ds['intz_diffu_2d'] * ds.dxCu, 'Y', boundary='fill')
                 + grid.diff(ds['intz_diffv_2d'] * ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
    BPT['Curl_Hdiff'] = Curl_Hdiff2
    
    Curl_Cor2 = ( - grid.diff((ds['intz_CAu_2d'] - ds['intz_gKEu_2d'] - ds['intz_rvxv_2d'])* ds.dxCu, 'Y', boundary='fill')
               + grid.diff((ds['intz_CAv_2d'] - ds['intz_gKEv_2d'] - ds['intz_rvxu_2d'])* ds.dyCv, 'X', boundary='fill'))/ ds.areacello_bu
    BPT['Curl_Cor'] = Curl_Cor2
    
    tmpx = (ds['hf_dudt_2d'] * colh_u - ds['intz_CAu_2d']-ds['intz_PFu_2d']-ds['intz_diffu_2d']-
           ds['intz_u_BT_accel_2d'] - ds['taux']/rho_0 + ds['taux_bot']/rho_0)
    tmpy = (ds['hf_dvdt_2d'] * colh_v - ds['intz_CAv_2d']-ds['intz_PFv_2d']-ds['intz_diffv_2d']-
           ds['intz_v_BT_accel_2d'] - ds['tauy'] /rho_0 + ds['tauy_bot']/rho_0)
    Curl_remap = ( - grid.diff(tmpx * ds.dxCu, 'Y', boundary='fill')
                 + grid.diff(tmpy * ds.dyCv, 'X', boundary='fill') )/ ds.areacello_bu
    BPT['Curl_remap'] = Curl_remap
    
    # Save dataset

    ds_save = xr.Dataset()

    ds_save['beta_V'] = BPT['vmo_bv']
    ds_save['beta_V'].attrs['units'] = "m/s^2"
    ds_save['beta_V'].attrs['standard_name'] = "Meridional Coriolis gradient x depth-integrated meridional velocity"
    
    ds_save['BPT'] = BPT['BPT'] + BPT['Curl_Cor'] + BPT['vmo_bv'] + BPT['Qm'] - BPT['fdhdt']
    ds_save['BPT'].attrs['units'] = "m/s^2"
    ds_save['BPT'].attrs['standard_name'] = "Bottom Pressure Torque"
    
    ds_save['Curl_Adv'] = (BPT['Curl_NL'] + BPT['Curl_remap'])
    ds_save['Curl_Adv'].attrs['units'] = "m/s^2"
    ds_save['Curl_Adv'].attrs['standard_name'] = "Curl of depth-integrated nonlinear advetion term"
    
    ds_save['Curl_taus'] =  BPT['Curl_taus']
    ds_save['Curl_taus'].attrs['units'] = "m/s^2"
    ds_save['Curl_taus'].attrs['standard_name'] = "Curl of Surface Wind Stress / rho_0"
    
    ds_save['Curl_taub'] =  BPT['Curl_taub']
    ds_save['Curl_taub'].attrs['units'] = "m/s^2"
    ds_save['Curl_taub'].attrs['standard_name'] = "- Curl of bottom boundary stress / rho_0"
    
    ds_save['Curl_diff'] = BPT['Curl_Hdiff']
    ds_save['Curl_diff'].attrs['units'] = "m/s^2"
    ds_save['Curl_diff'].attrs['standard_name'] = "Curl of depth-integrated horizontal diffusion"
    
    ds_save['Mass_flux'] = (- BPT['Qm'])
    ds_save['Mass_flux'].attrs['units'] = "m/s^2"
    ds_save['Mass_flux'].attrs['standard_name'] = " - Coriolis x Surface mass flux / rho_0"
    
    ds_save['eta_dt'] = BPT['fdhdt']
    ds_save['eta_dt'].attrs['units'] = "m/s^2"
    ds_save['eta_dt'].attrs['standard_name'] = " Coriolis x d(eta)/dt"
    
    ds_save['Curl_dudt'] = (-BPT['Curl_dudt'])
    ds_save['Curl_dudt'].attrs['units'] = "m/s^2"
    ds_save['Curl_dudt'].attrs['standard_name'] = " - Curl of depth-integrated du/dt"
    
    ds_save = xr.merge([ds_save, ds_grid])
    
    ds_save = ds_save.transpose('yq','yh','xq','xh')
    
    return ds_save


In [3]:
# Read hgrid data for proper lats/lons in plotting

hgrid_p25 = xr.open_dataset("../../../Data/Data_CM4_GFDL/CM4X_hgrid/ocean_hgrid_p25.nc")
hgrid_p125 = xr.open_dataset("../../../Data/Data_CM4_GFDL/CM4X_hgrid/ocean_hgrid_p125.nc")

In [5]:
%%time 

# Save CX4-p25 vorticity budget data

ds = xr.open_mfdataset("../../../Data/Data_CM4_GFDL/CM4X_p25/*p25.nc")

ds_grid = xr.open_dataset("../../../Data/Data_CM4_GFDL/CM4X_p25/ocean_monthly.static.nc")

ds = xr.merge([ds, ds_grid])

ds_vort = Vorticity_Budget(ds.isel(xq=slice(1, len(ds['xq'])), yq = slice(1, len(ds['yq']))), 
                           ds_grid.isel(xq=slice(1, len(ds['xq'])), yq = slice(1, len(ds['yq']))))

ds_vort = ds_vort.drop(['geolon_c', 'geolat_c'])
ds_vort = ds_vort.assign_coords({'geolon_c': xr.DataArray(hgrid_p25['x'][0:-1:2,0:-1:2].data, dims=["yq", "xq"]),
                        'geolat_c': xr.DataArray(hgrid_p25['y'][0:-1:2,0:-1:2].data, dims=["yq", "xq"])})

ds = ds.compute()
ds_vort = ds_vort.compute()

ds_vort.to_netcdf("../../../Data/Data_CM4_GFDL/CM4X_datashare/CM4X_p25_Vorticity_Budget.nc")
ds.to_netcdf("../../../Data/Data_CM4_GFDL/CM4X_datashare/CM4X_p25_Momentum_Budget.nc")


CPU times: user 2.31 s, sys: 724 ms, total: 3.03 s
Wall time: 2.62 s


In [7]:
%%time 

# Save CX4-p125 vorticity budget data
# coordinate values of vars and grid variables are not aligned properly
# so we need to account for this issue in .isel command.

ds = xr.open_mfdataset("../../../Data/Data_CM4_GFDL/CM4X_p125/*p125.nc")

ds_grid = xr.open_dataset("../../../Data/Data_CM4_GFDL/CM4X_p125/ocean_monthly.static.nc")

ds = xr.merge([ds.isel(xq=slice(1, len(ds['xq'])), yq = slice(1, len(ds['yq']))), 
               ds_grid.isel(xq=slice(0, len(ds['xq'])-1), yq = slice(0, len(ds['yq'])-1))])

ds_vort = Vorticity_Budget(ds, ds_grid.isel(xq=slice(0, len(ds_grid['xq'])-1), 
                                            yq = slice(0, len(ds_grid['yq'])-1)))

ds_vort = ds_vort.drop(['geolon_c', 'geolat_c'])
ds_vort = ds_vort.assign_coords({'geolon_c': xr.DataArray(hgrid_p125['x'][0:-1:2,0:-1:2].data, dims=["yq", "xq"]),
                        'geolat_c': xr.DataArray(hgrid_p125['y'][0:-1:2,0:-1:2].data, dims=["yq", "xq"])})

ds = ds.compute()
ds_vort = ds_vort.compute()

ds_vort.to_netcdf("../../../Data/Data_CM4_GFDL/CM4X_datashare/CM4X_p125_Vorticity_Budget.nc")
ds.to_netcdf("../../../Data/Data_CM4_GFDL/CM4X_datashare/CM4X_p125_Momentum_Budget.nc")



CPU times: user 11.4 s, sys: 4.72 s, total: 16.1 s
Wall time: 15.3 s
